In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats
import statsmodels.api as sm

from warnings import filterwarnings
filterwarnings('ignore')


pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth',None)

In [2]:
df = pd.read_csv('Lc club data for imputation.csv')
df.head()

,Unnamed: 0,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim
0,0,1077501,1296599,5000,5000,4975.0,36 months,10.65,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077501,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-85,1.0,NaN,3.0,0.0,13648,83.7,9.0,f,0.0,0.0,5861.071414,5831.78,5000.00,861.07,0.00,0.00,0.00,Jan-15,171.62,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
1,1,1077430,1314167,2500,2500,2500.0,60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-11,Default,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077430,car,bike,309xx,GA,1.00,0.0,Apr-99,5.0,NaN,3.0,0.0,1687,9.4,4.0,f,0.0,0.0,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-13,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
2,2,1077175,1313524,2400,2400,2400.0,36 months,15.96,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1077175,small_business,real estate business,606xx,IL,8.72,0.0,Nov-01,2.0,NaN,2.0,0.0,2956,98.5,10.0,f,0.0,0.0,3003.653644,3003.65,2400.00,603.65,0.00,0.00,0.00,Jun-14,649.91,Jan-16,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
3,3,1076863,1277178,10000,10000,10000.0,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1076863,other,personel,917xx,CA,20.00,0.0,Feb-96,1.0,35.0,10.0,0.0,5598,21.0,37.0,f,0.0,0.0,12226.302210,12226.30,10000.00,2209.33,16.97,0.00,0.00,Jan-15,357.48,Jan-15,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN
4,4,1075269,1311441,5000,5000,5000.0,36 months,7.90,156.46,A,A4,Veolia Transportaton,3 years,RENT,36000.0,Source Verified,Dec-11,Fully Paid,n,https://www.lendingclub.com/browse/loanDetail.action?loan_id=1075269,wedding,My wedding loan I promise to pay back,852xx,AZ,11.20,0.0,Nov-04,3.0,NaN,9.0,0.0,7963,28.3,12.0,f,0.0,0.0,5631.377753,5631.38,5000.00,631.38,0.00,0.00,0.00,Jan-15,161.03,Sep-15,0.0,1,INDIVIDUAL,0.0,NaN,NaN,NaN


In [4]:
from sklearn.impute import MissingIndicator
from sklearn.impute import KNNImputer
#from sklearn.impute import IterativeImputer
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer


In [5]:
df['emp_title']

0                          NaN
1                        Ryder
2                          NaN
3          AIR RESOURCES BOARD
4         Veolia Transportaton
                  ...         
254185              supervisor
254186     Coordinator of RSVP
254187                 Painter
254188         Lead Custodian 
254189                     NaN
Name: emp_title, Length: 254190, dtype: object

In [7]:
df['emp_title'].value_counts().nlargest(50)

emp_title
Manager                     1959
Teacher                     1927
Supervisor                   863
Registered Nurse             848
RN                           844
Sales                        685
Project Manager              670
manager                      590
Office Manager               571
Driver                       543
Director                     508
Engineer                     471
Owner                        468
US Army                      458
General Manager              450
teacher                      396
driver                       384
Vice President               380
Accountant                   358
Bank of America              347
Operations Manager           345
Administrative Assistant     332
supervisor                   322
Police Officer               318
Account Manager              314
Executive Assistant          313
Attorney                     298
Analyst                      294
President                    282
sales                        274


In [10]:
mv = pd.DataFrame({'total_null_values':df.isnull().sum(),'% null':df.isnull().sum()/df.shape[0] * 100})
mv = mv.sort_values(by = 'total_null_values',ascending = False)
mv.head(10)

,total_null_values,% null
mths_since_last_delinq,141446,55.645777
tot_coll_amt,63726,25.070223
total_rev_hi_lim,63726,25.070223
tot_cur_bal,63726,25.070223
emp_title,14010,5.511625
emp_length,9968,3.921476
last_pymnt_d,536,0.210866
revol_util,200,0.078681
collections_12_mths_ex_med,56,0.022031
last_credit_pull_d,21,0.008262


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254190 entries, 0 to 254189
Data columns (total 54 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  254190 non-null  int64  
 1   id                          254190 non-null  int64  
 2   member_id                   254190 non-null  int64  
 3   loan_amnt                   254190 non-null  int64  
 4   funded_amnt                 254190 non-null  int64  
 5   funded_amnt_inv             254190 non-null  float64
 6   term                        254190 non-null  object 
 7   int_rate                    254190 non-null  float64
 8   installment                 254190 non-null  float64
 9   grade                       254190 non-null  object 
 10  sub_grade                   254190 non-null  object 
 11  emp_title                   240180 non-null  object 
 12  emp_length                  244222 non-null  object 
 13  home_ownership

In [12]:
df.emp_title.sample(5)

164155     Sales & Service Representative
70446     LSUHSC-EA CONWAY MEDICAL CENTER
217401                pharmacy technician
205969    Transportation Engineer Level 2
19378                     Lockheed Martin
Name: emp_title, dtype: object

In [37]:
data = {}
value = []
for col in df.columns:
    value.append(df[col].sample(5))
    data[col] = df[col].sample(5)
    #value.clear()
                 
    
    

In [47]:
# creating a dictionary, having 'variable' as key and their top 5 'unique_values' as value, to use this dictionary in our datatype dataframe

unique_values_dict = {}
for i in df.columns:
    unique_values = list(pd.Series(df[i].unique()))
    uv = unique_values[:5]
    #lis1.append(unique_values[:5])
    unique_values_dict[i] = uv

In [46]:
pd.Series(data.keys())

0                     Unnamed: 0
1                             id
2                      member_id
3                      loan_amnt
4                    funded_amnt
5                funded_amnt_inv
6                           term
7                       int_rate
8                    installment
9                          grade
10                     sub_grade
11                     emp_title
12                    emp_length
13                home_ownership
14                    annual_inc
15           verification_status
16                       issue_d
17                   loan_status
18                    pymnt_plan
19                           url
20                       purpose
21                         title
22                      zip_code
23                    addr_state
24                           dti
25                   delinq_2yrs
26              earliest_cr_line
27                inq_last_6mths
28        mths_since_last_delinq
29                      open_acc
30        

In [45]:
pd.DataFrame(data = [data.keys()],columns = ['column'])

ValueError: 1 columns passed, passed data had 54 columns

In [39]:
data.get('id')

95875      3367060
40947      9656008
168954    31106170
240644    46154752
131409     1488454
Name: id, dtype: int64

In [22]:
pd.Series(data.keys())

0                     Unnamed: 0
1                             id
2                      member_id
3                      loan_amnt
4                    funded_amnt
5                funded_amnt_inv
6                           term
7                       int_rate
8                    installment
9                          grade
10                     sub_grade
11                     emp_title
12                    emp_length
13                home_ownership
14                    annual_inc
15           verification_status
16                       issue_d
17                   loan_status
18                    pymnt_plan
19                           url
20                       purpose
21                         title
22                      zip_code
23                    addr_state
24                           dti
25                   delinq_2yrs
26              earliest_cr_line
27                inq_last_6mths
28        mths_since_last_delinq
29                      open_acc
30        

In [31]:
value =[1,2,3,5,6]

In [32]:
value.clear()

In [33]:
value

[]